In [1]:
import gurobipy as gp
import numpy as np
from utilities import *
from model.tripartite_lp import GNNPolicy as Trimodel
from model.rgcn2 import GNNPolicy as RGCNmodel
from uc_class import UC
from torch_geometric.data import HeteroData
import torch
import json
from pytorch_metric_learning import losses
from pytorch_metric_learning.distances import DotProductSimilarity

#手动在连续开连续关中选择松弛掉的变量，构造低质量的解/还需要求解后验证
#或者写成一个鲁棒问题外层最大化函数值，内层是松弛的subMIP问题

In [2]:
uc_path = '/home/peilun/predict_heuristicFix_loopEnhance/UC_AF/80_c11_based_8_std.mod'
uc = UC(uc_path)

tri_path = 'model_save/1bin_80_c11_based_8_std_tripartite_lpsolve/model_best_acc.pth'
tri_model = Trimodel()
tri_dict = torch.load(tri_path)
tri_model.load_state_dict(tri_dict)
tri_model.eval()


# rgcn_path = 'model_save/1bin_80_c11_based_8_std_rgcn2/model_best.pth'
# rgcn_model = RGCNmodel()
# rgcn_dict = torch.load(rgcn_path)
# rgcn_model.load_state_dict(rgcn_dict)
# rgcn_model.eval()
print()

In [3]:
def mycallback(model, where):
    
    if where == gp.GRB.Callback.MIPSOL:
        t = time.time() - model._start_time
        obj = model.cbGet(gp.GRB.Callback.MIPSOL_OBJ)
        # objbnd = model.cbGet(GRB.Callback.MIPSOL_OBJBND)
        
        sol = model.cbGetSolution(model._u)
        gap = (model._obj - obj) / obj
        # print(t,obj,gap)
        # print(f"时刻：{t:.2f}, 与初始解距离{dif(model._sol, sol):.2f}, 与初始值gap{gap*100:.2f}%")
        # Add the time and objective to the lists
        model._times.append(t)
        model._objs.append(obj)
        # print(sol)
        model._sols.append(sol)
        # return sol,obj

In [4]:
def dif(a1,a2):
    l = len(a1)
    cnt=0
    for i in range(l):
        if a1[i] != a2[i]:
            cnt+=1
    return cnt/l
def gap(a1,a2):
    return (a2-a1)/max(a1,a2)

# def neighbor(sol1, sol2):
#     return np.bitwise_xor(sol1, sol2)

In [5]:
def solve(m,sol):
    m1 = m.copy()
    # N=sol.shape[0]
    # T=sol.shape[1]
    u = m1.getVars()[:len(sol)]
    for i in range(len(sol)):
        m1.addConstr(u[i]==sol[i],name=f'fix_u[{i}]')
    m1.optimize()
    return m1.objVal

In [6]:
def local_search(m,sol,relaxed):
    m1 = m.copy()
    u = m1.getVars()[:len(sol)]
    for i in range(len(sol)):
        if relaxed[i] == 0:
            m1.addConstr(u[i]==sol[i],name=f'fix_u[{i}]')
    m1.optimize()
    return m1.objVal

In [7]:
def get_positive(m,num_postive):
    m.setParam('OutputFlag', 0)
    m.setParam('MIPGap', 1e-7)
    m.setParam('PoolSearchMode', 2)
    m.setParam('PoolSolutions', num_postive)
    m.setParam('TimeLimit', 60)

    u=m.getVars()[:len(m.getVars())//3]
    #设置初始解
    # for i in range(len(u)):
    #     u[i].setAttr('Start', solt[i])
    m.optimize(mycallback)
    if m.SolCount > 0:
        positives = []
        for sn in range(m.SolCount):
            m.setParam('SolutionNumber', sn)
            sol = np.round(m.getAttr("Xn", u)).astype(int)
            positives.append(sol)
        positives = np.array(positives)
        return positives
    return np.array([])

In [8]:
def get_negtive(m,num_negtive):
    m.setParam('OutputFlag',0)
    m.setParam('MIPGap', 1e-2)
    m.setParam('PoolSearchMode', 2)
    m.setParam('PoolSolutions', num_negtive)
    m.setParam('TimeLimit', 4)
    m.optimize()

    if m.SolCount > 0:
        negatives = []
        for sn in range(m.SolCount):
            m.setParam('SolutionNumber',sn)
            u=m.getVars()[:len(m.getVars())//3]
            sol = np.round(m.getAttr("Xn", u)).astype(int)
            negatives.append(sol)
        negatives=np.array(negatives)
        return negatives
    return np.array([])

In [9]:
def relax_solve(m,sol,relax_percent):
    t=time.time()
    # m.write('ori.lp')
    u = m.getVars()[:len(sol)]
    relax_num = int(len(u)*relax_percent)
    alphas = m.addVars(len(sol),vtype=GRB.CONTINUOUS,name=f'alpha')
    for i in range(len(sol)):
        m.addConstr(alphas[i] >= u[i] - sol[i], name=f'alpha_up_{i}')
        m.addConstr(alphas[i] >= sol[i] - u[i], name=f'alpha_dowm{i}')
    m.addConstr(gp.quicksum(alphas[i] for i in range(len(alphas))) <= relax_num, name = 'relax_num')
    # m.write('relax.lp')
    m.optimize()
    return time.time()-t, m.status

In [10]:
# #测试对solt进行扰动，看原本数据是否适用于扰动数据，若可以则能提高生成数据效率
# log=False
# json_test_path = 'datasets/80_c11_based_8_std/json/test'
# json_test_files = os.listdir(json_test_path)[:1]

# shut_down = uc.Pishutdown
# startup = uc.Pistartup
# on = uc.ThTime_on_min
# off = uc.ThTime_off_min
# pmax = uc.ThPimax
# pmin = uc.ThPimin
# ramp_up = uc.Piup
# ramp_down = uc.Pidown

# for file in json_test_files:
#     # os.path.join(json_test_path, file)
#     with open(os.path.join(json_test_path, file),'r') as f:
#         data = json.load(f)
#     print(file)
#     pmax = uc.ThPimax
#     Dt = data['Dt']
#     Spin = data['Spin']
#     u0 = data['u0']
#     p0 = data['p0']
#     on_off = data['on_off']
#     m = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
    

#     Tri_graph=get_Tripartite_graph_lp(m)


#     solt = tri_model(Tri_graph).detach().cpu().numpy().reshape(-1,24)
    

#     solt1 = restore(solt,uc,data).reshape(-1,24).astype(int)
#     # print((solt1==solt).all())
#     print(id(solt),id(solt1))
#     # objt = solve(m,solt)
#     # negtive_datasets = get_negtive(m,solt,data['obj'],10,objt)
#     # postive_datasets = get_postive(m,solt,10)
#     # pos_effect = []

#     # for postive in postive_datasets:
#     #     obj = local_search(m,solt,postive)
#     #     pos_effect.append((objt-obj)/(objt-data['obj']))
#     # if len(pos_effect) == 0:
#     #     print('pos_effect: None')
#     # else:
#     #     print('pos_effect:',sum(pos_effect)/len(pos_effect))
    
    
#     # neg_effect = []

#     # for negtive in negtive_datasets:
#     #     obj = local_search(m,solt,negtive)
#     #     neg_effect.append((objt-obj)/(objt-data['obj']))
#     # if len(neg_effect) == 0:
#     #     print('neg_effect: None')
#     # else:
#     #     print('neg_effect:',sum(neg_effect)/len(neg_effect))
#         # print('neg_effect:',neg_effect)
    

In [12]:
log=False
json_test_path = 'datasets/80_c11_based_8_std/total_json/test'
json_test_files = os.listdir(json_test_path)[20:21]

shut_down = uc.Pishutdown
startup = uc.Pistartup
on = uc.ThTime_on_min
off = uc.ThTime_off_min
pmax = uc.ThPimax
pmin = uc.ThPimin
ramp_up = uc.Piup
ramp_down = uc.Pidown

for file in json_test_files:
    os.path.join(json_test_path, file)
    with open(os.path.join(json_test_path, file),'r') as f:
        data = json.load(f)

    pmax = uc.ThPimax
    Dt = data['Dt']
    Spin = data['Spin']
    u0 = data['u0']
    p0 = data['p0']
    on_off = data['on_off']
    m = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
    
    Tri_graph=get_Tripartite_graph_lp(m)
    # # rgcn_graph_o=get_graph_old(m1)
    # rgcn_graph=get_graph(m)

    solt = tri_model(Tri_graph).detach().cpu().numpy().reshape(-1,24)
    solt = restore(solt,uc,data).reshape(-1).astype(int)
    objt = solve(m,solt)
    u=m.getVars()[:len(m.getVars())//3]
    
    #设置初始解
    # for i in range(len(u)):
    #     u[i].setAttr('Start', solt[i])
    m._u = u
    m._obj = data['obj']
    m._sol = solt
    m._times=[]
    m._objs=[]
    m._sols=[]
    m._start_time = time.time()
    positive_datasets = get_positive(m,10)
    negative_datasets = get_negtive(m,10)
    # print(m._objs)
    for local in positive_datasets:
        print(local_search(m,solt,local))

3756046.020744723
3756046.0207447223
3754281.1005390063
3754281.1005390063
3756046.020744723
3754731.1005390063
3756046.0205219937
3756046.020744723
3756046.020521994
3756046.0205219937


In [14]:
for local in positive_datasets:
    print(dif(positive_datasets[0],local))

0.0
0.003125
0.03854166666666667
0.03958333333333333
0.023958333333333335
0.035416666666666666
0.035416666666666666
0.019791666666666666
0.019791666666666666
0.0265625


In [16]:
i=0
n=3
for i in range(3):
    pass
print(i)

3
